# Generate Europa Metadata

The Europa data release has two different types of products: (1) image mosaics and (2) individual L2 images. Within each of these products, different flavors of the data are being released. Some are photometrically trimmed and some are not.

- /jupiter/europa/galileo_voyager/controlled_mosaics - /scratch/ARD/stac/jupiter/europa/mosaics
- /jupiter/europa/galileo_voyager/controlled_images -  /scratch/ARD/stac/jupiter/europa/individual_images/*/

In [1]:
import glob
import json
import os
import requests

from amg.isismetadata import IsisFootPrintBlob
from amg.fgdcmetadata import FGDCMetadata, OrthographicFgdcParser
from amg.gdalmetadata import GDALMetadata
from amg.databasemetadata import DbMetadata, GenericSQLite
from amg.plaintextmetadata import PcAlignMetadata
from amg.formatters.stac_formatter import to_stac
from amg.formatters.fgdc_formatter import to_fgdc
from amg.utils import find_files, write_fgdc, write_stac
from amg import UnifiedMetadata

## Step I: Stage the data

## Step II: Build the STAC Collections

### Mosaics

Data are in `/scratch/jlaura/europa_mosaics/GLL_Obs` to be uploaded. Omit the .wkt files since they are image footprints.

In [2]:
EUROPA_MOSAICS = '/scratch/jlaura/europa_mosaics/GLL_Obs/COGs/'
OUTFILE = '2021_6_30_mosaics.lis'

In [3]:
stac_files = []
def create_unified_metadata_obj(image, footprint_filename, template, proj='equirect'):
    fgdc = FGDCMetadata(template, proj=proj)
    gd = GDALMetadata(image)
    ifb = IsisFootPrintBlob(footprint_filename)

    idname = os.path.basename(image)
    
    productname = os.path.splitext(os.path.basename(image))[0].replace('-cog', '')
    
    # Define overrides
    overrides = {'license': 'PDDL-1.0',
                 'missions':['Voyager 1', 'Voyager 2', 'Galileo'],
                 'doi':'https://doi.org/10.5066/P9VKKK7C',
                 'instruments':['Voyager I ISS, Voyager II ISS, Galileo SSI'],
                 'href':'https://asc-jupiter.s3-us-west-2.amazonaws.com/europa/galileo_voyager/controlled_mosaics',
                 'longitude_domain':360,
                 'projjson': None,
                 'title':f'Absolutely Controlled Image Mosaic; Europa; {productname}',
                 'targets':['Europa'],
                 'id':idname
}

    # Define mappings
    mappings = {'bbox':IsisFootPrintBlob, }
    
    record = UnifiedMetadata([fgdc, gd, ifb], overrides=overrides, mappings=mappings)
    
    return record

template = '../templates/europa_equirec_mosaics_fgdc_template.xml'

for i, fname in enumerate(glob.glob(EUROPA_MOSAICS + '/*.tif')): 
    basedir = os.path.dirname(fname)
        
    footprint_filename = fname.replace('-cog.tif', '.cub.wkt')
    if not os.path.exists(footprint_filename):
        print(f'No footprint for {fname}')
        continue
    
    
    # Data are in mixed projections, parse filenames to determine projection
    if 'Equi' in fname:
        proj = 'equirect'
    elif 'SPola' in fname or 'NPola' in fname:
        proj = 'polarst'
    else:
        print('Unknown projection', fname)
    
    # This builds the metadata record
    record = create_unified_metadata_obj(fname, footprint_filename, template, proj=proj)
    
    # Generate and write FGDC
    data_name = os.path.splitext(fname)[0]
    write_fgdc(os.path.join(basedir, data_name + '.xml'), to_fgdc(record))
    
    # Manually define the assets for this set of products
    assets = [{'title':'Image Thumbnail',
       'href':'{href}/{productid}.jpeg',
       'type':'image/jpeg',
       'roles':['thumbnail'],
       'key':'thumbnail'},
      {'title': 'Controlled Image',
       'href':'{href}/{productid}.tif',
       'type':'image/tiff; application=geotiff; profile=cloud-optimized',
       'roles':['data'],
       'key':'image'},
      {'title': 'FGDC Metadata',
       'href':'{href}/{productid}.xml',
       'type':'application/xml',
       'roles':['metadata'],
       'key':'fgdc_metadata'}
      ]

    
    stac_record = to_stac(record, 
                          assets=assets, 
                          extensions=["https://stac-extensions.github.io/projection/v1.0.0/schema.json",
                                      "https://stac-extensions.github.io/datacube/v1.0.0/schema.json",
                                      "https://raw.githubusercontent.com/thareUSGS/ssys/main/json-schema/schema.json"
                                     ]
                         )
    stac_file = os.path.join(basedir, data_name + '.json')
    write_stac(stac_file, stac_record)
    stac_files.append(stac_file)

# Write the list of created STAC files to the outfile
with open(os.path.join(EUROPA_MOSAICS, OUTFILE), 'w') as f:
    for stac_file in stac_files:
        f.write(stac_file + '\n')
print('Done!')

/work/users/jlaura/github/automated-metadata-generation/amg/__init__.py:72: UserWarning: Unable to find the attribute "horizontal_accuracy_report" in any data source.
  warnings.warn(f'Unable to find the attribute "{attr}" in any data source.')
/work/users/jlaura/github/automated-metadata-generation/amg/__init__.py:72: UserWarning: Unable to find the attribute "vertical_accuracy_report" in any data source.
  warnings.warn(f'Unable to find the attribute "{attr}" in any data source.')
/work/users/jlaura/github/automated-metadata-generation/amg/__init__.py:72: UserWarning: Unable to find the attribute "processing_environment" in any data source.
  warnings.warn(f'Unable to find the attribute "{attr}" in any data source.')


Done!


### Individual Images

In [4]:
EUROPA_L2 = '/scratch/ARD/stac/jupiter/europa/individual_images/'
OUTFILE = '2021_6_30_individual.lis'
db = '/work/projects/EuropaBasemap/lweller/GLL_VGR_Network/DB/Europa_GalileoMrg_Voyager_Final.db'

In [8]:
stac_files = []
def create_unified_metadata_obj(image, footprint_filename, template, proj='equirect'):
    fgdc = FGDCMetadata(template, proj=proj)
    gd = GDALMetadata(image)
    ifb = IsisFootPrintBlob(footprint_filename)

    # Setup to read a generic SQLite database
    db = '/work/projects/EuropaBasemap/lweller/GLL_VGR_Network/DB/Europa_GalileoMrg_Voyager_Final.db'
    fname = os.path.basename(image).split('.')[0]
    query = f"""SELECT * from geometry where geometry.File LIKE '{fname}%'"""
    column_remapper = {'StartTime':'start_date',
                       'EndTime':'stop_date',
                       'EmissionAngle':'emission_angle',
                       'IncidenceAngle':'incidence_angle',
                       'NorthAzimuth':'north_azimuth',
                       'SubSolarGroundAzimuth': 'subsolar_ground_azimuth'}
    gsql = GenericSQLite(db, sql=query, column_remapper=column_remapper)

    
    # Setup a second generic SQLite reader for mission/instrument information
    query = f"""SELECT * from image where image.File LIKE '{fname}%'"""
    column_remapper = {'SpacecraftName':'missions',
                       'InstrumentId':'instruments'}
    gsql2 = GenericSQLite(db, sql=query, column_remapper=column_remapper)
    
    productname = os.path.splitext(os.path.basename(image))[0].replace('-cog', '')
    
    # Define overrides
    overrides = {'license': 'PDDL-1.0',
                 'doi':'https://doi.org/10.5066/P9VKKK7C',
                 'href':'https://asc-jupiter.s3-us-west-2.amazonaws.com/europa/galileo_voyager/controlled_images',
                 'longitude_domain':360,
                 'title':f'Absolutely Controlled Image Mosaic; Europa; {productname}',
                 'targets':['Europa']}

    # Define mappings
    mappings = {'bbox':IsisFootPrintBlob, 'start_date':GenericSQLite, 'stop_date':GenericSQLite}
    
    record = UnifiedMetadata([fgdc, gd, ifb, gsql, gsql2], overrides=overrides, mappings=mappings)
    
    return record

template = '../templates/europa_individual_l2_fgdc.xml'

for i, fname in enumerate(glob.glob(EUROPA_L2 + '/*/*.tif')): 
    basedir = os.path.dirname(fname)
    footprint_filename = fname.replace('-cog.tif', '.cub.wkt')
    
    if not os.path.exists(footprint_filename):
        print(f'No footprint for {fname}')
        continue
    
    # Data are in mixed projections, parse filenames to determine projection
    if 'equi' in fname:
        proj = 'equirect'
    elif 'spola' in fname or 'npola' in fname:
        proj = 'polarst'
    else:
        print('Unknown projection', fname)
    
    # This builds the metadata record
    record = create_unified_metadata_obj(fname, footprint_filename, template, proj=proj)
        
    # Generate and write FGDC
    data_name = os.path.splitext(fname)[0]
    #write_fgdc(os.path.join(basedir, data_name + '.xml'), to_fgdc(record))
    
    # Manually define the assets for this set of products
    assets = [{'title':'Image Thumbnail',
       'href':'{href}/{productid}.jpeg',
       'type':'image/jpeg',
       'roles':['thumbnail'],
       'key':'thumbnail'},
      {'title': 'Controlled Image',
       'href':'{href}/{productid}.tif',
       'type':'image/tiff; application=geotiff; profile=cloud-optimized',
       'roles':['data'],
       'key':'image'},
      {'title': 'FGDC Metadata',
       'href':'{href}/{productid}.xml',
       'type':'application/xml',
       'roles':['metadata'],
       'key':'fgdc_metadata'},
      {'title': 'Supplemental XML Metadata',
       'href': '{href}/{productid}.tif.aux.xml',
       'type':'text/plain',
       'roles': ['metadata'],
       'key':'pds_label'}
      ]

    
    stac_record = to_stac(record, assets=assets, extensions=["https://stac-extensions.github.io/projection/v1.0.0/schema.json",
                                                             "https://stac-extensions.github.io/datacube/v1.0.0/schema.json",
                                                             "https://stac-extensions.github.io/view/v1.0.0/schema.json",
                                                             "https://raw.githubusercontent.com/thareUSGS/ssys/main/json-schema/schema.json"])
    stac_file = os.path.join(basedir, data_name + '.json')
    write_stac(stac_file, stac_record)
    stac_files.append(stac_file)
    
# Write the list of created STAC files to the outfile
with open(os.path.join(EUROPA_L2, OUTFILE), 'w') as f:
    for stac_file in stac_files:
        f.write(stac_file + '\n')
print('Done!')

Done!


In [1]:
import pystac

In [6]:
from pystac.extensions.base import ItemExtension


ImportError: cannot import name 'ItemExtension' from 'pystac.extensions.base' (/home/jlaura/anaconda3/envs/amg/lib/python3.8/site-packages/pystac/extensions/base.py)

In [2]:
from pystac.extensions.base import ItemExtension

ImportError: cannot import name 'ItemExtension' from 'pystac.extensions.base' (/home/jlaura/anaconda3/envs/amg/lib/python3.8/site-packages/pystac/extensions/base.py)